# Training the model
**Table of Contents**
1. [Setup](#1-setup)
2. [Dataset Preparation](#2-dataset-preparation)
3. [Training Loop](#3-training-loop)


## 1. Setup

In [1]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [2]:
####################
# Required Modules #
####################

# Generic/Built-in
import random

# Libs
import torch
import numpy as np
from torch.utils.data import DataLoader

# Custom
from src.data_preparation import *
from src.models import *
from src.train_eval import *

In [3]:
# Seeding
SEED = 42

# To be safe, seed all modules for full reproducibility
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # If using CUDA
np.random.seed(SEED)
random.seed(SEED)

## 2. Dataset Preparation

In [4]:
train_dataset, val_dataset, test_dataset, norm_stats = prepare_datasets(
    sequence_size=1000,
    stride=25,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=SEED,
    load_if_exists=True
)

✅ HARDataset object, normalization statistics, and split indices loaded from saved_components
✅ Created train, validation, and test datasets (normalized to train set)


## 3. Training Loop

In [5]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 5

In [6]:
# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
validation_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

In [7]:
# Load model
model = HarTransformer(
    input_size=6,
    hidden_size=30,
    num_layers=2,
    num_heads=2,
    dropout_prob=0.1,
    num_classes=7,
)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

HarTransformer model loaded on cuda.


In [8]:
training_loss_history, validation_loss_history, accuracy_history, f1_history, precision_history, recall_history = train_HAR70_model(
    model, 
    optimizer, 
    train_dataloader, 
    validation_dataloader, 
    num_epochs = num_epochs
)

Epoch [1/5] | Time: 204.18s
(Training) Loss: 0.0039
(Validation) Loss: 0.0027, Accuracy: 0.5615, F1: 0.5502, Precision: 0.6816, Recall: 0.5615
Epoch [2/5] | Time: 189.72s
(Training) Loss: 0.0025
(Validation) Loss: 0.0022, Accuracy: 0.5938, F1: 0.6109, Precision: 0.6974, Recall: 0.5938
Epoch [3/5] | Time: 190.13s
(Training) Loss: 0.0022
(Validation) Loss: 0.0020, Accuracy: 0.6017, F1: 0.6071, Precision: 0.6465, Recall: 0.6017
Epoch [4/5] | Time: 190.16s
(Training) Loss: 0.0021
(Validation) Loss: 0.0021, Accuracy: 0.6733, F1: 0.6437, Precision: 0.6570, Recall: 0.6733
Epoch [5/5] | Time: 191.71s
(Training) Loss: 0.0020
(Validation) Loss: 0.0019, Accuracy: 0.6741, F1: 0.6805, Precision: 0.6954, Recall: 0.6741
✅ Model saved: models\HarTransformer_2025-03-12_22-11-47\HarTransformer_epoch5.pth


In [11]:
save_training_plots_and_metric_history(training_loss_history, validation_loss_history, accuracy_history, f1_history, precision_history, recall_history, type(model).__name__)

✅ Plots saved to: results\HarGRU_2025-03-12_00-00-38
✅ Metric histories saved to: results\HarGRU_2025-03-12_00-00-38\metric_histories.pth


In [12]:
# Test metrics
loss, accuracy, f1, precision, recall, conf_matrix = evaluate_HAR70_model(model, test_dataloader)
print(f"(Test) Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

(Test) Loss: 0.0021, Accuracy: 0.6007, F1: 0.6197, Precision: 0.7078, Recall: 0.6007
